In [3]:
# let's define a few helper utilities

from pathlib import Path
import hashlib
import subprocess
import json
from bs4 import BeautifulSoup
from functools import wraps
import inspect


text_cache = Path('cache')

def sha1(input_string):
    """Helper to hash input strings"""
    try:

        # Step 5: Create a new SHA-1 hash object
        hash_object = hashlib.sha1()

        # Step 6: Update the hash object with the bytes-like object
        hash_object.update(input_string.encode('utf-8'))

        # Step 7: Get the hexadecimal representation of the hash
        return hash_object.hexdigest()
    except Exception as e:
        raise ValueError(input_string) from e


from functools import wraps
import inspect
import json
import pandas as pd
import hashlib


class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if hasattr(obj, 'to_dict'):
            return obj.to_dict()
        if isinstance(obj, pd.Int64Dtype):
            return int(obj)  # Convert Int64 to a regular int
        return json.JSONEncoder.default(self, obj)


def stored(func):
    """
    implements nix-like durable memoisation of function results.

    Lazy way to avoid recomputing expensive calls. Expects results to be JSON-serializable
    """
    @wraps(func)
    def CACHE(*args, **kwargs):
        name = func.__name__
        meta = {}

        meta["name"] = name
        meta["func"] = inspect.getsource(func)
        meta["args"] = args
        meta["kwargs"] = kwargs

        js = json.dumps(meta, cls=CustomEncoder)  # Using CustomEncoder
        sha = hashlib.sha1(js.encode('utf-8'))

        digest = sha.hexdigest()

        path = text_cache / f"{digest}-{name}.json"

        if path.exists():
            with path.open('r') as r:
                cached = json.load(r)
            return cached["result"]
        result = func(*args, **kwargs)
        meta["result"] = result
        with path.open('w') as w:
            json.dump(meta, w, cls=CustomEncoder)  # Using CustomEncoder
        return result

    return CACHE

In [4]:
from pydantic import BaseModel, Field, HttpUrl
from typing import List
import configparser

# Define Pydantic models for each config section
class YouTrackConfig(BaseModel):
    Token: str
    BaseUrl: str

class VsemRabotaConfig(BaseModel):
    ProjectId: str
    WBTeam: List[str] = Field(default_factory=list)
    Name: str

# Main configuration model that includes both sections
class AppConfig(BaseModel):
    YouTrack: YouTrackConfig
    VsemRabota: VsemRabotaConfig

# Parse the INI file
def parse_config_to_model(config_file: str) -> AppConfig:
    config = configparser.ConfigParser()
    config.read(config_file)
    
    # YouTrack section
    youtrack_token = config.get('YouTrack', 'Token').strip("'")
    youtrack_base_url = config.get('YouTrack', 'BaseUrl')
    
    # VsemRabota Config section
    project_id = config.get('VsemRabota Config', 'ProjectId')
    name = config.get('VsemRabota Config', 'Name')
    wb_team = config.get('VsemRabota Config', 'WBTeam').split(', ')
    
    # Create instances of the models
    youtrack_config = YouTrackConfig(Token=youtrack_token, BaseUrl=youtrack_base_url)
    vsem_rabota_config = VsemRabotaConfig(ProjectId=project_id, WBTeam=wb_team, Name=name)
    
    # Aggregate into the main config model
    app_config = AppConfig(YouTrack=youtrack_config, VsemRabota=vsem_rabota_config)
    
    return app_config

In [5]:
config_file = 'yt.ini'  # Update this path
config_model = parse_config_to_model(config_file)
print(config_model)

YouTrack=YouTrackConfig(Token='perm:VmFsZW50aW5fTWFyY2h1aw==.NjktMzE1.IgVAwvLdgn8H1dpNCX2FOS9T0Yhdpn', BaseUrl='https://youtrack.wildberries.ru') VsemRabota=VsemRabotaConfig(ProjectId='77-167', WBTeam=['Valentin_Marchuk', 'Liventsev_sergei'], Name='VsemRabota')


In [ ]:
import requests

headers = {
    'Authorization': f'Bearer {config_model.YouTrack.Token}',
    'Content-Type': 'application/json',
}

search_query = 'Business Line:HR   State: Resolved sort by: {Threat Score} desc'

response = requests.get(f'{config_model.YouTrack.Token}/api/issues?query={search_query}', headers=headers)

### Client

In [176]:
import requests
from urllib.parse import urlencode

class YouTrackAPIError(Exception):
    """Exception raised for errors in the YouTrack API."""
    def __init__(self, status_code, message):
        super().__init__(f"HTTP {status_code}: {message}")
        self.status_code = status_code
        self.message = message

class YouTrackClient:
    def __init__(self, config):
        self.config = config
        self.headers = {
            'Authorization': f'Bearer {self.config.Token}',
            'Content-Type': 'application/json',
        }

    @stored
    def get(self, endpoint: str, params: dict = None) -> dict:
        """Sends a GET request to a specified endpoint with optional query parameters."""
        url = f"{self.config.BaseUrl}{endpoint}"
        # param_str = '&'.join([f'{key}={value}' for key, value in params.items()])
        if params:
            url += f"?{urlencode(params)}"
        try:
            response = requests.get(url, headers=self.headers, timeout=300)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as e:
            raise YouTrackAPIError(response.status_code, response.text) from e

    def create_issue_with_custom_fields(self, issue_custom_fields, project_id, summary, description):
        serialized_fields = issue_custom_fields.serialize()
        
        data = {
            "project": {"id": project_id},
            "summary": summary,
            "description": description,
            "customFields": serialized_fields,
        }

        return self.post('/api/issues', data=data)

    def post(self, endpoint: str, data: dict = None) -> dict:
        """Sends a POST request to a specified endpoint with optional JSON data."""
        url = f"{self.config.BaseUrl}{endpoint}"
        try:
            response = requests.post(url, json=data, headers=self.headers)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.HTTPError as e:
            raise YouTrackAPIError(response.status_code, response.text) from e

    def get_issues(self, search_query: str, additional_params: dict = None) -> dict:
        """
        Retrieves issues from YouTrack based on a search query and additional parameters.

        :param search_query: A string representing the search query for filtering issues.
        :param additional_params: Optional dictionary of additional query parameters.
        :return: A dictionary containing the API response with issues.
        """
        params = {'query': search_query}
        if additional_params:
            params.update(additional_params)
        
        return self.get('/api/issues', params=params)

    def get_issue(self, issue_id, fields=None):
        issue_fields = [
            'id', 'idReadable', 'summary', 'description', 'reporter(id,login,fullName)',
            'created', 'updated', 'resolved', 'priority', 'state', 'assignee(id,login,fullName)',
            'tags(name)', 'comments(text,author(fullName),created)', 'attachments(name,url)',
            'voters(login,fullName)', 'watchers(login,fullName)', 'sprints(name)',
            'customFields($type,id,projectCustomField($type,id,field($type,id,name)),value($type,avatarUrl,buildLink,color(id),fullName,id,isResolved,localizedName,login,minutes,name,presentation,text))',
            'linkType(name,sourceToTarget,targetToSource)',
            'issues(id,idReadable,summary)'
        ]
        
        if fields is None:
            fields = 'id,idReadable,summary,description,customFields(id,projectCustomField(field(name))),linkType(name,sourceToTarget,targetToSource),issues(id,idReadable,summary)'
            fields = ','.join(issue_fields)

        params = {
            'fields': fields
        }

        return self.get(f'/api/issues/{issue_id}', params=params)

    def get_field(self, issue_id, field_id):
        params = {'fields': 'id,projectCustomField(id,field(id,name)),value(id,isResolved,localizedName,name)'}
        return self.get(f'/api/issues/{issue_id}/customFields/{field_id}', params=params)

    def get_issue_history(self, issue_id):
        params = {
            'categories': 'CustomFieldCategory,CommentsCategory',
            'fields': 'author(name,login),timestamp,target(text),added(name),removed(name)'
        }
        return self.get(f'/api/issues/{issue_id}/activities', params=params)

    def get_projects(self, additional_params: dict = None):
        leader_fields = ['login', 'name', 'id']
        createdby_fields = ['login', 'name', 'id']
        
        params = {'fields': 'id,name,shortName,createdBy(login,name,id),leader(login,name,id),key'}
        if additional_params:
            params.update(additional_params)
        
        return self.get('/api/admin/projects', params=params)

    def get_link(self, issue_id):
        return self.get(f'/api/issues/{issue_id}/links?fields=linkType(name,sourceToTarget,targetToSource),issues(id,idReadable,summary)')

    def get_project_by_name(self, shortName: str):
        params = {
            'fields': 'id,name,shortName',
            'query': shortName
        }

        return self.get('/api/admin/projects', params=params)

    def to_dict(self): 
        return {
            "version": "v1"
        }
    

### Initialize

In [177]:
youtrack_client = YouTrackClient(config=config_model.YouTrack)
        

In [ ]:
import json


resp = youtrack_client.get_issue('VR-4118')



print(json.dumps(resp, indent=2, ensure_ascii=False))


In [ ]:
youtrack_client.get_link('92-770926')

In [206]:
issues_info = []
issue_history = {}
issue_links = {}

In [181]:
with open('/issues_info.json', 'w') as f:
    f.write(json.dumps(issues_info, indent=2, ensure_ascii=False))

In [ ]:
with open('issues_info.json', 'w') as f:
    f.write(json.dumps(issues_info, indent=2, ensure_ascii=False))

In [194]:
import json
from datetime import datetime


def save_files(issues_info, issue_history, issue_links):
    now = datetime.now()
    formatted_time = now.strftime("%Y%m%d%H%M")
    
    # Write to the file
    with open(f"./datasets/issues_info_{formatted_time}.json", 'w') as f:
        f.write(json.dumps(issues_info, indent=2, ensure_ascii=False))
    
    
    with open(f"./datasets/issue_history_{formatted_time}.json", 'w') as f:
        f.write(json.dumps(issue_history, indent=2, ensure_ascii=False))

    with open(f"./datasets/issue_links_{formatted_time}.json", 'w') as f:
        f.write(json.dumps(issue_links, indent=2, ensure_ascii=False))

In [208]:
from tqdm import tqdm

query1 = '(Subtask of: VR-4118) or (Subtask of: (Subtask of: VR-4118)) or issue id: VR-4118'
query2 = 'created by: Mikhail_Shogin, Valentin_Marchuk, Vansevich.E, grigorev.mark, Vitaliy_Guselnikov, krupenko.ilya, aleksandrov.e23'
query3 = 'Project: VsemRabota order by: created by'
query4 = 'Project: {HR ВЕБ}, {DevOps HR}, {VRT}  order by: created by'

@stored
def get_issues(query, skip: int, top: int):
    return youtrack_client.get_issues(query, {'$skip': skip, '$top': top })

@stored
def get_issue(id):
    return youtrack_client.get_issue(id)

@stored
def get_issue_history(id):
    return youtrack_client.get_issue_history(id)

@stored
def get_links(id):
    return youtrack_client.get_link(id)


issues = get_issues(query4, 0, top=15000)

for issue in tqdm(issues):
    resp = get_issue(issue['id'])
    
    try:
        issues_info.append(resp)
    
        # print(f'Processing {resp['idReadable']}')
        if resp['idReadable'] not in ['VR-4665', 'VR-4664']:
            history = get_issue_history(issue['id'])
        
            issue_history[issue['id']] = history
    
        issue_links[issue['id']] = get_links(issue['id'])
    except Exception as exc:
        print(f'Exception: {resp['idReadable']}')


save_files(issues_info, issue_history, issue_links)

    

100%|████████████████████████████████████████████████████████████████████████████████████████████████| 13556/13556 [2:09:52<00:00,  1.74it/s]


In [ ]:
resp

In [ ]:
import pandas as pd

df = pd.read_json('./datasets/issue_history_202406070110.json')

In [195]:
save_files(issues_info, issue_history, issue_links)


In [167]:
data = issue_history

In [168]:
def find_structure(data):
    structures = {}
    for key, activities in data.items():
        for activity in activities:
            for change_type in ['added', 'removed']:
                change_items = activity.get(change_type, None)
                if isinstance(change_items, list):
                    for item in change_items:
                        if isinstance(item, dict):
                            struct_type = item['$type']
                            if struct_type not in structures:
                                structures[struct_type] = {'fields': set(item.keys())}
                            else:
                                structures[struct_type]['fields'].update(item.keys())
                else:
                    struct_type = type(change_items).__name__
                    if struct_type not in structures:
                        structures[struct_type] = {'fields': {change_type}}
                    else:
                        structures[struct_type]['fields'].add(change_type)
    return structures

structures = find_structure(data)

for struct_type, info in structures.items():
    print(f"Type: {struct_type}")
    print(f"Fields: {', '.join(info['fields'])}")
    print()

Type: StateBundleElement
Fields: $type, name

Type: IssueComment
Fields: $type

Type: EnumBundleElement
Fields: $type, name

Type: int
Fields: added, removed

Type: NoneType
Fields: added, removed

Type: User
Fields: $type, name



In [ ]:
issue_history

In [104]:
issue_history.items()[0]

TypeError: 'dict_items' object is not subscriptable

In [122]:
# Function to extract and expand records
def expand_records(issue_id, activities):
    records = []
    for activity in activities:
        base_record = {
            'issue_id': issue_id,
            'timestamp': activity['timestamp'],
            'author_login': activity['author']['login'],
            'author_name': activity['author']['name'],
            'activity_type': activity['$type']
        }

        # Expand added items
        if isinstance(activity.get('added'), list):
            for added_item in activity['added']:
                record = base_record.copy()
                record.update({
                    'added_type': added_item.get('$type'),
                    'added_name': added_item.get('name'),
                    'removed_type': None,
                    'removed_name': None,
                })
                records.append(record)
        elif isinstance(activity.get('added'), int):
            record = base_record.copy()
            record.update({
                'added_type': 'int',
                'added_name': activity['added'],
                'removed_type': None,
                'removed_name': None,
            })
            records.append(record)
        elif activity.get('added') is None:
            record = base_record.copy()
            record.update({
                'added_type': 'NoneType',
                'added_name': None,
                'removed_type': None,
                'removed_name': None,
            })
            records.append(record)

        # Expand removed items
        if isinstance(activity.get('removed'), list):
            for removed_item in activity['removed']:
                record = base_record.copy()
                record.update({
                    'removed_type': removed_item.get('$type'),
                    'removed_name': removed_item.get('name'),
                    'added_type': None,
                    'added_name': None,
                })
                records.append(record)
        elif isinstance(activity.get('removed'), int):
            record = base_record.copy()
            record.update({
                'removed_type': 'int',
                'removed_name': activity['removed'],
                'added_type': None,
                'added_name': None,
            })
            records.append(record)
        elif activity.get('removed') is None:
            record = base_record.copy()
            record.update({
                'removed_type': 'NoneType',
                'removed_name': None,
                'added_type': None,
                'added_name': None,
            })
            records.append(record)

        # If no added or removed, add the base record
        if not activity.get('added') and not activity.get('removed'):
            record = base_record.copy()
            record.update({
                'added_type': None,
                'added_name': None,
                'removed_type': None,
                'removed_name': None,
            })
            records.append(record)
    return records

# Convert the data to a list of dictionaries suitable for a DataFrame
all_records = []
for issue_id, activities in data.items():
    all_records.extend(expand_records(issue_id, activities))

# Create the DataFrame
df = pd.DataFrame(all_records)

In [128]:
df[df['added_type'] == 'StateBundleElement']['added_name'].unique()

array(['Blocked', 'Canceled', 'Review', 'To Do', 'Done', 'In Progress',
       'Completed', 'Decomposition', 'Incomplete'], dtype=object)

In [126]:
df.to_csv('issues_history.csv')

In [129]:
df.dtypes

issue_id         object
timestamp         int64
author_login     object
author_name      object
activity_type    object
added_type       object
added_name       object
removed_type     object
removed_name     object
dtype: object

In [125]:
df

,issue_id,timestamp,author_login,author_name,activity_type,added_type,added_name,removed_type,removed_name
0,92-1335094,1715079625547,Lavrinenko.V2,Лавриненко Василий Александрович,CustomFieldActivityItem,StateBundleElement,Blocked,None,None
1,92-1335094,1715079625547,Lavrinenko.V2,Лавриненко Василий Александрович,CustomFieldActivityItem,None,None,StateBundleElement,To Do
2,92-1335094,1715079657376,Lavrinenko.V2,Лавриненко Василий Александрович,CommentActivityItem,IssueComment,None,None,None
3,92-1335094,1717056369984,Lavrinenko.V2,Лавриненко Василий Александрович,CommentActivityItem,IssueComment,None,None,None
4,92-1335094,1717056374479,Lavrinenko.V2,Лавриненко Василий Александрович,CustomFieldActivityItem,StateBundleElement,Canceled,None,None
...,...,...,...,...,...,...,...,...,...
18097,92-1291639,1713362471864,workflow_user_1224751877,YouTrack Workflow,CustomFieldActivityItem,None,None,int,180
18098,92-1291639,1713366070130,workflow_user_1224751877,YouTrack Workflow,CustomFieldActivityItem,int,300,None,None
18099,92-1291639,1713366070130,workflow_user_1224751877,YouTrack Workflow,CustomFieldActivityItem,None,None,int,240
18100,92-1291639,1713368719576,Vansevich.E,Вансевич Евгений Алексеевич,CustomFieldActivityItem,StateBundleElement,Done,None,None


In [101]:
df.loc[0]['added']

[{'name': 'Blocked', '$type': 'StateBundleElement'}]

In [ ]:
youtrack_client.get_issue(issues[2]['id'])

youtrack_client.get_issue_history(issues[2]['id'])

In [ ]:
# youtrack_client.get_issue(issues[2]['id'])

In [ ]:
issues_info[0]

In [141]:
data = resp


issue_details = []

for data in issues_info:
    # Extract main issue details
    issue_details.append({
        'issue_id': data['id'],
        'idReadable': data['idReadable'],
        'updated': data['updated'],
        'resolved': data['resolved'],
        'reporter_login': data['reporter']['login'],
        'reporter_fullName': data['reporter']['fullName'],
        'reporter_id': data['reporter']['id'],
        'created': data['created'],
        'summary': data['summary'],
        'description': data['description'],
    })

custom_fields = []

for issue in issues_info:
    data = issue
    for field in data['customFields']:
        field_name = field['projectCustomField']['field']['name']
        field_value = field.get('value', {})
        if isinstance(field_value, dict):
            for sub_key, sub_value in field_value.items():
                custom_fields.append({
                    'issue_id': issue['id'],
                    'field_name': field_name,
                    'attribute': sub_key,
                    'value': sub_value
                })
        else:
            custom_fields.append({
                'issue_id': issue['id'],
                'field_name': field_name,
                'attribute': 'value',
                'value': field_value
            })

# Convert to DataFrames
df_issue_details = pd.DataFrame(issue_details)
df_custom_fields = pd.DataFrame(custom_fields)

In [142]:
df_issue_details

,issue_id,idReadable,updated,resolved,reporter_login,reporter_fullName,reporter_id,created,summary,description
0,92-1335094,VR-10298,1717056374474,1.717056e+12,Valentin_Marchuk,Марчук Валентин,24-5060,1714994891129,[FE] Аналитка и декомпозиция. AuthV3 для Adminka,Нужно декомпозировать логику фронта для переез...
1,92-1150577,VR-8574,1717056303583,NaN,Lavrinenko.V2,Лавриненко Василий Александрович,24-1498,1708613415913,[FE] Монорепа. Переход на AuthV3,Сваггер <https://gitlab.wildberries.ru/wbx-tea...
2,92-770926,VR-4118,1717019595766,NaN,Livencev.Sergey,Ливенцев Сергей Викторович,24-1839,1685604329877,[Epic.Technology.Infra] Все продукты. Переход ...,# Проблема\n\nЕсть необходимость обновить сист...
3,92-1393585,VR-10905,1716985371341,1.716985e+12,zyuvanov.sergey,Зюванов Сергей Игоревич,24-6045,1716897681148,[BUG] Не подтягивается количество сотрудников ...,# Общее описание\n\nОкружение: `prod`\nУчётна...
4,92-884030,VR-5423,1716965163704,1.716965e+12,Yaraslau_Chaplinski,Чаплинский Ярослав Эдуардович,24-3942,1695099712908,[BE] - AuthV3 - Интеграция auth v3 в loans-api,# Проблема\n\nИБ требуют от нас переезда на но...
...,...,...,...,...,...,...,...,...,...,...
87,92-1230688,VR-9287,1714992466361,1.714721e+12,Valentin_Marchuk,Марчук Валентин,24-5060,1711459122629,[UserStory] Интеграция AuthV3 в сервисы C&B,## Проблема\n\nИБ требуют от нас переезда на н...
88,92-1220975,VR-9188,1714992388923,1.711351e+12,Valentin_Marchuk,Марчук Валентин,24-5060,1711099588878,[BE] Поддержать работу фронта и AuthV3,Block от @Lavrinenko.V2\n\n**Таска фронтов:** ...
89,92-1334165,VR-10280,1714989253516,1.714989e+12,Valentin_Marchuk,Марчук Валентин,24-5060,1714987420209,[FE] Декомпозировать задачи по переезду Анкеты...,"Нарезать тасок, которые осталось сделать для т..."
90,92-1280441,VR-9773,1714987683902,1.713429e+12,Valentin_Marchuk,Марчук Валентин,24-5060,1712931789941,[BE] - AuthV3 - Интеграция auth v3 в worksheet...,# Задача #1\n\ncrm-front ходит в сервис worksh...


In [143]:
df_custom_fields

,issue_id,field_name,attribute,value
0,92-1335094,Type,localizedName,None
1,92-1335094,Type,color,"{'id': '0', '$type': 'FieldStyle'}"
2,92-1335094,Type,name,Task
3,92-1335094,Type,id,53-3712
4,92-1335094,Type,$type,EnumBundleElement
...,...,...,...,...
5183,92-1291639,Environments,$type,EnumBundleElement
5184,92-1291639,Services,value,"[{'localizedName': None, 'color': {'id': '0', ..."
5185,92-1291639,Start Date,value,1711713600000
5186,92-1291639,Due Date,value,1713355200000


In [149]:
issues_history_df = df

df.dtypes

issue_id         object
timestamp         int64
author_login     object
author_name      object
activity_type    object
added_type       object
added_name       object
removed_type     object
removed_name     object
dtype: object

In [145]:
df_custom_fields.dtypes

issue_id      object
field_name    object
attribute     object
value         object
dtype: object

In [146]:
df_issue_details.dtypes

issue_id              object
idReadable            object
updated                int64
resolved             float64
reporter_login        object
reporter_fullName     object
reporter_id           object
created                int64
summary               object
description           object
dtype: object

In [144]:
df_custom_fields.to_csv('custom_fields.csv')
df_issue_details.to_csv('issue_details.csv')

In [160]:
# Extract Assignee and Reviewer [HR] fields
assignee_fields = df_custom_fields[df_custom_fields['field_name'] == 'Assignee']
reviewer_fields = df_custom_fields[df_custom_fields['field_name'] == 'Reviewer [HR]']

# Pivot the fields to get one row per issue_id with separate columns for Assignee and Reviewer
assignee_pivot = assignee_fields.pivot(index='issue_id', columns='attribute', values='value').reset_index()
reviewer_pivot = reviewer_fields.pivot(index='issue_id', columns='attribute', values='value').reset_index()

# Merge the pivoted data with the issues_history DataFrame
issues_history_with_assignee = issues_history_df.merge(assignee_pivot[['issue_id', 'name']], on='issue_id', how='left')
issues_history_with_assignee_reviewer = issues_history_with_assignee.merge(reviewer_pivot[['issue_id', 'name']], on='issue_id', how='left', suffixes=('_assignee', '_reviewer'))

issues_history_with_assignee_reviewer

issues_history_with_assignee_reviewer.to_csv('issues_history.csv')

In [161]:
custom_fields = df_custom_fields
issue_details = df_issue_details

In [164]:
# Merge the custom fields (Assignee, Reviewer, Type, and State) with the issue_details DataFrame
issue_details_with_custom_fields = issue_details.merge(assignee_pivot_login[['issue_id', 'login']], on='issue_id', how='left')
issue_details_with_custom_fields = issue_details_with_custom_fields.merge(reviewer_pivot_login[['issue_id', 'login']], on='issue_id', how='left', suffixes=('_Assignee', '_Reviewer'))
issue_details_with_custom_fields = issue_details_with_custom_fields.merge(type_pivot[['issue_id', 'name']], on='issue_id', how='left')
issue_details_with_custom_fields = issue_details_with_custom_fields.merge(state_pivot[['issue_id', 'name']], on='issue_id', how='left', suffixes=('_Type', '_State'))


issue_details_with_custom_fields.head()

issue_details_with_custom_fields.to_csv('issues_details.csv')

issue_details_with_custom_fields

,issue_id,idReadable,updated,resolved,reporter_login,reporter_fullName,reporter_id,created,summary,description,login_Assignee,login_Reviewer,name_Type,name_State
0,92-1335094,VR-10298,1717056374474,1.717056e+12,Valentin_Marchuk,Марчук Валентин,24-5060,1714994891129,[FE] Аналитка и декомпозиция. AuthV3 для Adminka,Нужно декомпозировать логику фронта для переез...,Lavrinenko.V2,Valentin_Marchuk,Task,Canceled
1,92-1150577,VR-8574,1717056303583,NaN,Lavrinenko.V2,Лавриненко Василий Александрович,24-1498,1708613415913,[FE] Монорепа. Переход на AuthV3,Сваггер <https://gitlab.wildberries.ru/wbx-tea...,Lavrinenko.V2,Lavrinenko.V2,Task,Blocked
2,92-770926,VR-4118,1717019595766,NaN,Livencev.Sergey,Ливенцев Сергей Викторович,24-1839,1685604329877,[Epic.Technology.Infra] Все продукты. Переход ...,# Проблема\n\nЕсть необходимость обновить сист...,Mikhail_Shogin,NaN,Epic,In Progress
3,92-1393585,VR-10905,1716985371341,1.716985e+12,zyuvanov.sergey,Зюванов Сергей Игоревич,24-6045,1716897681148,[BUG] Не подтягивается количество сотрудников ...,# Общее описание\n\nОкружение: `prod`\nУчётна...,Markov.Kirill3,Markov.Kirill3,Bug,Done
4,92-884030,VR-5423,1716965163704,1.716965e+12,Yaraslau_Chaplinski,Чаплинский Ярослав Эдуардович,24-3942,1695099712908,[BE] - AuthV3 - Интеграция auth v3 в loans-api,# Проблема\n\nИБ требуют от нас переезда на но...,krupenko.ilya,grigorev.mark,Task,Done
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,92-1230688,VR-9287,1714992466361,1.714721e+12,Valentin_Marchuk,Марчук Валентин,24-5060,1711459122629,[UserStory] Интеграция AuthV3 в сервисы C&B,## Проблема\n\nИБ требуют от нас переезда на н...,Valentin_Marchuk,NaN,User Story,Canceled
88,92-1220975,VR-9188,1714992388923,1.711351e+12,Valentin_Marchuk,Марчук Валентин,24-5060,1711099588878,[BE] Поддержать работу фронта и AuthV3,Block от @Lavrinenko.V2\n\n**Таска фронтов:** ...,grigorev.mark,goncharov.v38,Task,Done
89,92-1334165,VR-10280,1714989253516,1.714989e+12,Valentin_Marchuk,Марчук Валентин,24-5060,1714987420209,[FE] Декомпозировать задачи по переезду Анкеты...,"Нарезать тасок, которые осталось сделать для т...",Lavrinenko.V2,Valentin_Marchuk,Task,Done
90,92-1280441,VR-9773,1714987683902,1.713429e+12,Valentin_Marchuk,Марчук Валентин,24-5060,1712931789941,[BE] - AuthV3 - Интеграция auth v3 в worksheet...,# Задача #1\n\ncrm-front ходит в сервис worksh...,krupenko.ilya,Yaraslau_Chaplinski,Task,Done


In [77]:
df_custom_fields[df_custom_fields['field_name'] == 'Estimation']['attribute'].unique()

array(['presentation', 'id', 'minutes', '$type'], dtype=object)

In [ ]:
df_custom_fields[(df_custom_fields['field_name'] == 'Estimation') & df_custom_fields]['minutes'].unique()

In [ ]:
next(info for info in issues_info if info['idReadable'] == 'VR-9855')

In [47]:
df_issue_details

,idReadable,updated,resolved,reporter_login,reporter_fullName,reporter_id,created,summary,description
0,VR-6857,1716908503845,None,Mikhail_Shogin,Шогин Михаил Михайлович,24-4442,1701123515809,[UserStory] Все продукты. Переход на внешний с...,# Проблема\n\nОбъявлено об обязательном перехо...


In [50]:
df_custom_fields[~(df_custom_fields['attribute'] == 'color')]

,field_name,attribute,value
0,Type,localizedName,None
2,Type,name,User Story
3,Type,id,53-3711
4,Type,$type,EnumBundleElement
5,Assignee,login,Valentin_Marchuk
6,Assignee,avatarUrl,/hub/api/rest/avatar/1641693e-0092-45e4-88a2-b...
7,Assignee,name,Марчук Валентин
8,Assignee,fullName,Марчук Валентин
9,Assignee,id,24-5060
10,Assignee,$type,User


In [19]:
youtrack_client.get_issue_history('VR-2934')

[{'removed': [],
  'added': [{'$type': 'IssueComment'}],
  'author': {'login': 'Pechnikov.D',
   'name': 'Печников Дмитрий Валерьевич',
   '$type': 'User'},
  'target': {'text': 'Не воспроизводится. \n\n```\n$ curl http://auth.alljobswb.svc.k8s.stage-dp/api/v2/auth/login_by_phone -X POST --data \'{"phone":"79876543211","is_terms_and_conditions_accepted":true,"scope":"crm"}\'\n{"token":"baf8f750db96451a540981b002db260587d25b0f5ca737479d864eb6d6848b72ce88a03925fd05009a0a01a719fde7e9d23b9e2bc6784a6d1a83ee74","till_next_request":120}\n\n$ curl http://auth.alljobswb.svc.k8s.stage-dp/api/v2/auth/login_by_phone -X POST --data \'{"phone":"79876543211","is_terms_and_conditions_accepted":true,"scope":"crm"}\'\n{"status":500,"code":15,"message":"0-UnknownError, 1-NotFound, 2-InvalidRequest, 3-AuthNoSessionCookie, 4-AuthServiceError, 5-IncorrectAccount(лицевой счет), 6-(лицевой счет будет добавлен после заведения контракта), 7-WBBalanceError, 8-NotEnoughMoney (запрашиваемая сумма больше чем есть н

In [ ]:
from enum import Enum
import json

class CustomFieldEnum(Enum):
    TYPE = "Type"
    DEVS = "Devs"
    ASSIGNEE = "Assignee"
    REVIEWER_HR = "Reviewer [HR]"
    PRIORITY = "Priority"
    COMPLEXITY = "Complexity"
    ESTIMATION = "Estimation"
    STATE = "State"
    STREAM = "Stream"
    TIME_SPENT = "Time Spent"
    PRODUCT = "Product"
    ENVIRONMENTS = "Environments"
    SERVICES = "Services"
    START_DATE = "Start Date"
    DUE_DATE = "Due Date"

In [ ]:
def get_customfield_value(customfield, issue):
    return next(r['value'] for r in issue['customFields'] if r['projectCustomField']['field']['name'] == customfield)

def get_issue_authority(issue):
    assignee = get_customfield_value('Assignee', issue)
    reviewer = get_customfield_value('Reviewer [HR]', issue)
    product = get_customfield_value('Product', issue)
    reporter = resp['reporter']['fullName']

    return {
        'assignee': assignee['fullName'] if assignee else None,
        'reviewer': reviewer['fullName'] if reviewer else None,
        'products': [p['name'] for p in product],
        'reporter': reporter
    }

In [14]:
get_customfield_value(CustomFieldEnum.PRIORITY.value, resp)

{'localizedName': None,
 'name': 'Major',
 'color': {'id': '4', '$type': 'FieldStyle'},
 'id': '53-2',
 '$type': 'EnumBundleElement'}

In [19]:
issues = youtrack_client.get_issues('Board HR Support Tasks: {Current Sprint}')

payload_issues = []
for issue in issues:
    issue = youtrack_client.get_issue(issues[0]['id'])
    payload_issues.append(issue)


In [ ]:
payload_issues

In [23]:
issue_history = youtrack_client.get_issue_history(issues[0]['id'])

In [ ]:
from datetime import datetime, timedelta


# Define enums for custom fields
class CustomFieldEnum(Enum):
    STATE = "State"
    ASSIGNEE = "Assignee"

# Helper functions
def parse_timestamp(timestamp):
    return datetime.fromtimestamp(timestamp / 1000)

def get_custom_field_value(issue, field_name_enum):
    custom_fields = issue.get('customFields', [])
    for field in custom_fields:
        field_name = field['projectCustomField']['field']['name']
        if field_name == field_name_enum.value:
            if field['value'] is None:
                return None
            elif isinstance(field['value'], list):  # For MultiEnum fields
                return [item['name'] for item in field['value']]
            else:  # For SingleEnum fields
                return field['value']['name']
    return None

def get_assignee(issue):
    assignee = get_custom_field_value(issue, CustomFieldEnum.ASSIGNEE)
    return assignee
    # print(assignee)
    # return assignee.get('login') if assignee else None

def get_state(issue):
    return get_custom_field_value(issue, CustomFieldEnum.STATE)

def is_issue_in_progress(issue):
    state = get_state(issue)
    return state == "In Progress"

def find_inactive_employees(client, search_query, inactivity_period_hours):
    issues = client.get_issues(search_query)
    payload_issues = [client.get_issue(issue['id']) for issue in issues]
    
    current_time = datetime.now()
    inactivity_threshold = current_time - timedelta(hours=inactivity_period_hours)

    active_assignees = set()
    last_active_time = {}

    for issue in payload_issues:
        issue_id = issue['id']
        assignee_login = get_assignee(issue)
        
        if assignee_login:
            if is_issue_in_progress(issue):
                active_assignees.add(assignee_login)
                print(issue)
                print('\n\n')
                last_active_time[assignee_login] = parse_timestamp(issue['created'])
        
        history = client.get_issue_history(issue_id)
        for activity in history:
            if activity['$type'] == 'CustomFieldActivityItem':
                added_items = activity.get('added')
                if added_items and isinstance(added_items, list):
                    if 'StateBundleElement' in [item['$type'] for item in added_items if isinstance(item, dict)]:
                        assignee_login = activity['author']['login']
                        timestamp = parse_timestamp(activity['timestamp'])
                        last_active_time[assignee_login] = timestamp

    inactive_employees = []
    for assignee, last_active in last_active_time.items():
        if last_active < inactivity_threshold and assignee not in active_assignees:
            inactive_employees.append(assignee)

    return inactive_employees



search_query = 'Board HR Support Tasks: {Current Sprint}'
inactivity_period_hours = 1

inactive_employees = find_inactive_employees(youtrack_client, search_query, inactivity_period_hours)
print("Inactive employees:", inactive_employees)

In [ ]:
issue = youtrack_client.get_issue(issues[0]['id'])


issue

In [ ]:
import pandas as pd

file_path = r'C:\Users\MGroup\Downloads\VULN tokens.csv'
df = pd.read_csv(file_path)

issues = list(dict.fromkeys(df['Ссылка на youtrack'].values.tolist()))
issues = [ i.replace('https://youtrack.wildberries.ru/issue/', '') for i in issues]
issues

In [ ]:
df

In [ ]:
issues_authority = []
for id in issues:
    issue = youtrack_client.get_issue(id)
    if issue is None:
        print(f'Issue {id} was not found.')
        continue
    issue_authority = get_issue_authority(issue)
    issues_authority.append({
         'issue_id': id,
        **issue_authority
    })


issues_authority

In [ ]:
df['issue_id'] = df['Ссылка на youtrack'].str.extract(r'issue/(VR-\d+)')

json_df = pd.DataFrame(issues_authority)
merged_df = pd.merge(df, json_df, on="issue_id", how="left")

merged_df.to_csv('vulns.csv')

In [ ]:
merged_df['products'].apply(assign_team)

In [ ]:

# Define a function to determine the team based on the product
def assign_team(products):
    if any(product in ['admin', 'vsemrabota', 'wbteam'] for product in products):
        return 'WB Team'
    elif 'ats' in products:
        return 'ATS Team'
    else:
        return 'Empty Team'

# Apply the function to create a new 'Team' column
merged_df['Team'] = merged_df['products'].apply(assign_team)

# Group by 'Team' and then by 'issue_id', and collect tokens
grouped_data = merged_df.groupby(['Team', 'issue_id'])['Секрет'].apply(list)

output_format = {}
for (team, issue_id), tokens in grouped_data.items():
    if team not in output_format:
        output_format[team] = []
    output_format[team].append((issue_id, tokens))

# Print in the desired format
for team, issues in output_format.items():
    print(f"{team}:")
    for issue_id, tokens in issues:
        print(f"{issue_id}")
        for token in tokens:
            print(f"- {token}")
    print("\n")

In [ ]:
merged_df.to_csv('vulns.csv')

In [ ]:
issues = youtrack_client.get('/api/issues', params={'query': 'Business Line:HR   State: Resolved sort by: {Threat Score} desc'})
print(issues)

In [ ]:
youtrack_client.get_issues('Business Line:HR   State: Resolved sort by: {Threat Score} desc')

In [13]:
def get_project_by_name(project_short_name: str):
    projects = youtrack_client.get_project_by_name(project_short_name)

    target_project = next(project for project in projects if project['shortName'] == project_short_name)
    return target_project
   
get_project_by_name('VR')

{'shortName': 'VR', 'name': 'VsemRabota', 'id': '77-167', '$type': 'Project'}

In [ ]:
import json

print(.dumps(youtrack_client.get_issue('VR-5418'), indent=2))

In [ ]:
resp = youtrack_client.get_issues('Reviewer [HR]: Valentin_Marchuk, Vansevich.E, Mikhail_Shogin, grigorev.mark , Vitaliy_Guselnikov, aleksandrov.e23, krupenko.ilya, mustafetov.n , goncharov.v38, korolev.artem14  and State: Review ')
resp

In [ ]:
review_items = []

for item in resp:
    print(item['id'])
    issue = youtrack_client.get_issue(item['id'])
    history = youtrack_client.get_issue_history(issue["idReadable"])
    
    latest_review_time, hours_passed = get_latest_review_time_and_passed_hours(history)
    
    reviewer = next(field['value']['login'] for field in issue['customFields'] if field['projectCustomField']['field']['name'] == 'Reviewer [HR]')
    
    review_items.append({
        'reviewer': reviewer,
        'issue_id': issue["idReadable"],
        'summary': issue["summary"],
        'latest_review_time': latest_review_time,
        'hours_passed': hours_passed
    })

In [ ]:
youtrack_client.get_field('VR-8939', '86-578')

In [ ]:
issue = youtrack_client.get_issue('92-1177453')
history = youtrack_client.get_issue_history(issue["idReadable"])

latest_review_time, hours_passed = get_latest_review_time_and_passed_hours(history)

reviewer = next(field['value']['login'] for field in issue['customFields'] if field['projectCustomField']['field']['name'] == 'Reviewer [HR]')

item = {
    'reviewer': reviewer,
    'issue_id': issue["idReadable"],
    'summary': issue["summary"],
    'latest_review_time': latest_review_time,
    'hours_passed': hours_passed
}
# print(f'[{issue["idReadable"]}] {issue["summary"]} {reviewer}')

item

In [ ]:
youtrack_client.get_issue('VR-8939')

In [ ]:
history = youtrack_client.get_issue_history('VR-8816')

history

In [ ]:
activity_log = history

In [ ]:
from datetime import datetime, timedelta

def get_latest_review_time_and_passed_hours(activity_log):
    # There was an error due to the first item in the log having 'added' as an integer. Let's correct this and process again.
    latest_review_timestamp = None
    for activity in activity_log:
        if 'added' in activity and isinstance(activity['added'], list) and any('name' in d and d['name'] == 'Review' for d in activity['added']):
            latest_review_timestamp = activity['timestamp']
    
    # Convert the timestamp from milliseconds to seconds
    latest_review_timestamp_seconds = latest_review_timestamp / 1000 if latest_review_timestamp else None
    
    # Calculate the hours passed from the latest status change to "Review" until now
    if latest_review_timestamp_seconds:
        latest_review_time = datetime.utcfromtimestamp(latest_review_timestamp_seconds)
        current_time = datetime.utcnow()
        hours_passed = (current_time - latest_review_time).total_seconds() / 3600
    else:
        hours_passed = None
    
    return latest_review_time, hours_passed

get_latest_review_time_and_passed_hours(activity_log)

In [ ]:
review_items

In [ ]:
from collections import defaultdict
from datetime import datetime
import re

def escape_markdown(text):
    # Define the pattern for special Markdown characters that need to be escaped
    special_chars_pattern = r'([_*\[\]()~`>#+\-=|{}.!])'
    # Replace each special character with its escaped version
    escaped_text = re.sub(special_chars_pattern, r'\\\1', text)
    return escaped_text

mapped_user_names = {
    'Valentin_Marchuk': 'valuamba',
    'Vansevich.E': 'evansevich',
    'Mikhail_Shogin': 'mshogin'
}

grouped_issues = defaultdict(list)
for issue in review_items:
    grouped_issues[issue['reviewer']].append(issue)

# "[*" + escapeMarkdown(issueStr) + "*]"  + "(" + issueUrl + ")\n\n"

# Sort and format output
output = "\#review\n\nЗадачи на ревью:\n\n"
for reviewer, issues in grouped_issues.items():
    output += f"@{mapped_user_names[reviewer]}:\n"
    # Sort issues by hours_passed
    sorted_issues = sorted(issues, key=lambda x: x['hours_passed'], reverse=True)
    for issue in sorted_issues:
        issue_url = f'https://youtrack.wildberries.ru/issue/{issue["issue_id"]}'
        time_passed = f"{int(issue['hours_passed'] / 24)}d" if issue['hours_passed'] > 24 else f"{round(issue['hours_passed'])}h"
        output += f"{time_passed} [*\[{escape_markdown(issue['issue_id'])}\]*]({issue_url}) {escape_markdown(issue['summary'])}\n"
    output += '\n'

print(output.strip())

In [ ]:
https://t.me/c/2011635320/2/3

In [ ]:
import asyncio
import telegram

test_chat = -1002011635320
test_thread = 2

# https://t.me/c/1593412877/10060/10061
bot = telegram.Bot("6779084548:AAEzWtAFQphlMonph8R-pw9IlU9YzFPal2k")


await bot.send_message(text=output.strip(), chat_id=-1001593412877, message_thread_id=10060, parse_mode='MarkdownV2')

### Custom Fields

In [6]:
class CustomField:
    def __init__(self, name, value, field_type="field_type"):
        self.name = name
        self.value = value
        self.field_type = field_type

class EnumField(CustomField):
    def __init__(self, name, value):
        super().__init__(name, {"name": value}, field_type="SingleEnumIssueCustomField")

class UserField(CustomField):
    def __init__(self, name, login):
        super().__init__(name, {"login": login}, field_type="SingleUserIssueCustomField")

class MultiEnumField(CustomField):
    def __init__(self, name, values):
        super().__init__(name, [{"name": value} for value in values], field_type="MultiEnumIssueCustomField")

class PeriodField(CustomField):
    def __init__(self, name, presentation):
        super().__init__(name, {"presentation": presentation}, field_type="PeriodIssueCustomField")

class DateField(CustomField):
    def __init__(self, name, value=None):
        super().__init__(name, value, field_type="DateIssueCustomField")

class SimpleField(CustomField):
    def __init__(self, name, value):
        super().__init__(name, value, field_type="SimpleIssueCustomField")


In [7]:
from datetime import datetime, timezone
import pytz

class VRIssueCustomFields:
    def __init__(self, assignee, type, devs, reviewer_hr, priority, state, stream, estimation, complexity, product, environments, services, start_date=None, due_date=None):
        self.assignee = assignee
        self.type = type
        self.devs = devs
        self.reviewer_hr = reviewer_hr
        self.priority = priority
        self.state = state
        self.stream = stream
        self.estimation = estimation
        self.complexity = complexity
        self.product = product
        self.environments = environments
        self.services = services
        self.start_date = self.convert_date_to_utc_millis(start_date)
        self.due_date = self.convert_date_to_utc_millis(due_date)

    def serialize(self):
        custom_fields = [
            EnumField("Type", self.type),
            EnumField("Devs", self.devs),
            UserField("Assignee", self.assignee),
            UserField("Reviewer [HR]", self.reviewer_hr),
            EnumField("Priority", self.priority),
            EnumField("State", self.state),
            EnumField("Stream", self.stream),
            PeriodField("Estimation", self.estimation),
            EnumField("Complexity", self.complexity),
            MultiEnumField("Product", [self.product]),
            EnumField("Environments", self.environments),
            MultiEnumField("Services", self.services.split(',')),
            DateField("Start Date", self.start_date),
            DateField("Due Date", self.due_date),
        ]

        # Filtering out None values and adjusting for field type
        serialized_fields = []
        for field in custom_fields:
            if field.value is not None:
                serialized_fields.append({"name": field.name, "value": field.value, "$type": field.field_type})
                # if isinstance(field, DateField):
                #     # Ensure date fields are serialized with proper value format
                #     serialized_fields.append({"name": field.name, "value": {"$type": "Long", "value": field.value}, "$type": field.field_type})
                # else:
                #     serialized_fields.append({"name": field.name, "value": field.value, "$type": field.field_type})
        
        return serialized_fields

    @staticmethod
    def convert_date_to_utc_millis(date_str):
        if date_str is None:
            return None
        # Assuming the input format is 'YYYY-MM-DD'
        dt = datetime.strptime(date_str, '%Y-%m-%d')
        # Adjust for UTC+3
        tz_utc_3 = pytz.timezone('Europe/Moscow')
        dt = tz_utc_3.localize(dt)
        # Convert to UTC
        dt_utc = dt.astimezone(pytz.utc)
        # Convert to milliseconds since epoch
        return int(dt_utc.timestamp() * 1000)

# Adapt EnumField, UserField, PeriodField, MultiEnumField, DateField, and SimpleField definitions as required


### Create Task

In [41]:
# Instantiate VRIssueCustomFields with hardcoded values
issue_custom_fields = VRIssueCustomFields(
    assignee="grigorev.mark",
    type="Task",
    devs="BE",
    reviewer_hr="Markov.Kirill3",
    priority="Show-stopper",
    state="Review",
    stream="Product",
    estimation="4h",
    complexity="Вроде не изян",
    product="all",
    environments="prod",
    services="auth",
    start_date=None,  # Assuming these fields are optional and can be None
    due_date='2024-05-19'
)

# To get the serialized form suitable for creating an issue in YouTrack
serialized_fields = issue_custom_fields.serialize()

# `serialized_fields` now contains the custom fields in the format expected by YouTrack API.


In [40]:
serialized_fields

[{'name': 'Type',
  'value': {'name': 'Task'},
  '$type': 'SingleEnumIssueCustomField'},
 {'name': 'Devs',
  'value': {'name': 'BE'},
  '$type': 'SingleEnumIssueCustomField'},
 {'name': 'Assignee',
  'value': {'login': 'grigorev.mark'},
  '$type': 'SingleUserIssueCustomField'},
 {'name': 'Reviewer [HR]',
  'value': {'login': 'Markov.Kirill3'},
  '$type': 'SingleUserIssueCustomField'},
 {'name': 'Priority',
  'value': {'name': 'Show-stopper'},
  '$type': 'SingleEnumIssueCustomField'},
 {'name': 'State',
  'value': {'name': 'Review'},
  '$type': 'SingleEnumIssueCustomField'},
 {'name': 'Stream',
  'value': {'name': 'Product'},
  '$type': 'SingleEnumIssueCustomField'},
 {'name': 'Estimation',
  'value': {'presentation': '4h'},
  '$type': 'PeriodIssueCustomField'},
 {'name': 'Complexity',
  'value': {'name': 'Вроде не изян'},
  '$type': 'SingleEnumIssueCustomField'},
 {'name': 'Product',
  'value': [{'name': 'all'}],
  '$type': 'MultiEnumIssueCustomField'},
 {'name': 'Environments',
  'val

In [46]:
youtrack_client = YouTrackClient(config=config_model.YouTrack)


In [47]:

# Usage example:
issue_custom_fields = VRIssueCustomFields(
    assignee="grigorev.mark",
    type="Task",
    devs="BE",
    reviewer_hr="Valentin_Marchuk",
    priority="Normal",
    state="Review",
    stream="Product",
    estimation="4h",
    complexity="Вроде не изян",
    product="all",
    environments="prod",
    services="auth",
    start_date=None,
    due_date='2024-05-19',
)

response_data = youtrack_client.create_issue_with_custom_fields(issue_custom_fields, "77-167", "Test 1", "Test 2")
print(response_data)


{'id': '92-1208267', '$type': 'Issue'}


In [ ]:
youtrack_client.get_issue('92-1208267')

## Retro

In [ ]:
'updated by: grigorev.mark updated: 2024-03-15 .. now'

In [8]:
"""Reviewer [HR]: Valentin_Marchuk, Vansevich.E, Mikhail_Shogin, grigorev.mark , 
Vitaliy_Guselnikov, aleksandrov.e23, krupenko.ilya, mustafetov.n , goncharov.v38, korolev.artem14  and State: Review
"""

'Reviewer [HR]: Valentin_Marchuk, Vansevich.E, Mikhail_Shogin, grigorev.mark , \nVitaliy_Guselnikov, aleksandrov.e23, krupenko.ilya, mustafetov.n , goncharov.v38, korolev.artem14  and State: Review\n'

In [ ]:
Valentin_Marchuk, Vansevich.E, grigorev.mark, Vitaliy_Guselnikov, aleksandrov.e23, krupenko.ilya, mustafetov.n, korolev.artem14

In [9]:
def get_activity_issues(login):
    resp = youtrack_client.get_issues(f'updated by: {login} updated: 2024-03-01 .. now')

    issues = []
    for item in resp:
        issue = youtrack_client.get_issue(item['id'])
        issues.append(issue)

    issuesWithStates = [
    {
        'idReadable': i["idReadable"],
        'summary': i["summary"],
        'state': next((i['value']['name'] for i in i['customFields'] 
                       if i['projectCustomField']['field']['name'] == 'State' and i['value'] and i['value']['name'] in ['To Do', 'Canceled', 'Done', 'Review', 'Blocked', 'Canceled', 'In Progress']), None)
    } for i in issues]

    return issuesWithStates

In [ ]:
print('\n'.join([f'[{i["idReadable"]}] {i["summary"]}' for i in issues]))

In [78]:
issuesWithStates = [
    {
        'idReadable': i["idReadable"],
        'summary': i["summary"],
        'state': next(i['value']['name'] for i in i['customFields'] if i['projectCustomField']['field']['name'] == 'State' and i['value'], None)
    }
        
for i in issues]

In [24]:
def print_issues(login, issues):
    print(f'\n\n------------------- {login} -------------------\n\n')
    state_importance = {
        'Blocked': (7, '🚫'),
        'To Do': (6, '📝'),
        'In Progress': (5, '🔨'),
        'Review': (4, '👀'),
        'Waiting Response': (3, '⏳'),
        'Done': (2, '✅'),
        'Canceled': (1, '❌')  # Added "Canceled" state with emoji
    }
    
    # Grouping issues by state and sorting
    grouped_issues = {}
    for issue in [i for i in issues if i['state']]:
        state = issue['state']
        if state in grouped_issues:
            grouped_issues[state].append(issue)
        else:
            grouped_issues[state] = [issue]
    
    # Sorting states by their importance
    sorted_states = sorted(grouped_issues.items(), key=lambda x: state_importance[x[0]][0], reverse=True)
    
    # Formatting output
    for state, issues in sorted_states:
        print(f"{state} {state_importance[state][1]}")
        for issue in issues:
            print(f"[{issue['idReadable']}] {issue['summary']}")
        print()  # Add an extra line between groups for readability

In [28]:
participants = [
    'korolev.artem14',
    'Vansevich.E',
    'grigorev.mark',
    'Vitaliy_Guselnikov',
    'Valentin_Marchuk',
    'hlopkov.mihail2',
    'aleksandrov.e23',
    'krupenko.ilya',
    'mustafetov.n',
    'goncharov.v38',
    'nevskiy.vilyam'
]

In [30]:
for login in participants[4:5]:
    issues = get_activity_issues(login)
    print_issues(login, issues)



------------------- Valentin_Marchuk -------------------


Blocked 🚫
[VR-8515] [BE] Реализовать маскировку в crm-api
[HRWEB-2378] [ARCH] WB Basket. Описание архитектуры
[VR-8146] [BE] Написать клиент для WB Basket и подключить к HR Scans
[VR-9035] Понять какие интеграции планируются
[VR-8662] [VULN-201] Race Condition на проверку кода при аутентификации по ОТП на всемработе
[HRWEB-2300] Настроить алертинг по паникам для ns: hr
[VR-6018] [VULN-15] CORS Misconfiguration на ресурсах всемработы

To Do 📝
[VR-9584] [BE] Внедрить взаимодействие с Я.Календарем на страницу
[VR-8991] [ARCH] Оргструктура. Обновление данных в kafka
[VR-9290] [UserStory] Интеграция AuthV3 в сервисы INFRA
[VR-9287] [UserStory] Интеграция AuthV3 в сервисы C&B
[VR-8407] [UserStory] MVP админки. Админка для L1 - hr-backoffice
[VR-8946] [Epic] Развитие компетенций инженеров и членов команды
[VR-9499] [UserStory] Развернуть datalake для будущей аналитики
[VR-9498] [Epic] Развернуть datalake для будущей аналитики
[VR-68

In [10]:
issues = get_activity_issues('grigorev.mark')

SSLError: HTTPSConnectionPool(host='youtrack.wildberries.ru', port=443): Max retries exceeded with url: /api/issues?query=updated+by%3A+grigorev.mark+updated%3A+2024-03-01+..+now (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))

In [22]:
issues

[{'idReadable': 'VR-9330',
  'summary': '[BE] - AuthV3 - Интеграция auth v3 в orgstruct-api',
  'state': None},
 {'idReadable': 'VR-9536',
  'summary': '[BE] - AuthV3 - Интеграция auth v3 в chat-service',
  'state': None},
 {'idReadable': 'HRWEB-2450',
  'summary': '[BE] Написать клиент WB Basket для s3-adapter',
  'state': None},
 {'idReadable': 'VR-9527',
  'summary': '[BE] Добавить ошибку - 403 - Unauthorized в мидлваре AuthV1()',
  'state': None},
 {'idReadable': 'VULN-4236',
  'summary': '[team.wb.ru] Доступ в чужой аккаунт',
  'state': None},
 {'idReadable': 'VR-9523',
  'summary': '[BE] Убрать возможность обмена jwt токена на CRM/WB  токены для пользователей не являющихся сотрудниками',
  'state': None},
 {'idReadable': 'VR-4118',
  'summary': '[TECH] Все продукты. Переход на внешний сервис авторизации: auth v.3',
  'state': None},
 {'idReadable': 'VR-8412',
  'summary': '[BE] - AuthV3 - Интеграция auth v3 в staff-api',
  'state': None},
 {'idReadable': 'VR-9132',
  'summary': '

In [ ]:
{'idReadable': 'VR-9124',
  'summary': '[BE] Добавить метод для обмена токенами в клиент auth',

In [ ]:
issues[0]

In [ ]:
{'projectCustomField': {'field': {'name': 'State',
     'id': '42-3',
     '$type': 'CustomField'},
    'id': '78-1996',
    '$type': 'StateProjectCustomField'},
   'value': {'isResolved': False,
    'localizedName': None,
    'name': 'In Progress',
    'color': {'id': '25', '$type': 'FieldStyle'},
    'id': '55-3717',
    '$type': 'StateBundleElement'},
   'id': '78-1996',
   '$type': 'StateIssueCustomField'},

In [69]:
issue = next(i for i in issues[0]['customFields'] if i['projectCustomField']['field']['name'] == 'State')

issue['value']['name']

'In Progress'

In [ ]:
issues[0]

{'field': {'name': 'State', 'id': '42-3', '$type': 'CustomField'},
 'id': '78-1996',
 '$type': 'StateProjectCustomField'}